In [1]:
import subprocess
import sys

# Get all installed packages
result = subprocess.run([sys.executable, '-m', 'pip', 'list'], 
                       capture_output=True, text=True)
print(result.stdout)

Package                           Version
--------------------------------- ------------------
absl-py                           2.3.0
aext-assistant                    4.20.0
aext-assistant-server             4.20.0
aext-core                         4.20.0
aext-core-server                  4.20.0
aext_environments_server          4.20.0
aext-panels                       4.20.0
aext-panels-server                4.20.0
aext-project-filebrowser-server   4.20.0
aext-share-notebook               4.20.0
aext-share-notebook-server        4.20.0
aext-shared                       4.20.0
aext-toolbox                      4.20.0
aiobotocore                       2.12.3
aiohappyeyeballs                  2.4.0
aiohttp                           3.10.5
aioitertools                      0.7.1
aiosignal                         1.2.0
alabaster                         0.7.16
alembic                           1.13.3
altair                            5.0.1
anaconda-anon-usage               0.4.4
anaconda-

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile
import json
import tensorflow as tf
import graphviz
from pathlib import Path
from tensorflow.keras.applications import Xception, DenseNet201
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Concatenate, Input
from tensorflow.keras.models import Model

In [27]:
kaggle_credentials = {
    "username":"stephenkoay",
    "key":"f3b44375dae5c5f27a6d28b2f235efaa"
}

In [28]:
kaggle_dir = os.path.expanduser("~/.kaggle")
os.makedirs(kaggle_dir, exist_ok=True)

In [29]:
with open(os.path.join(kaggle_dir, "kaggle.json"), 'w') as f:
    json.dump(kaggle_credentials, f)
print("Kaggle credentials saved successfully!")

Kaggle credentials saved successfully!


In [34]:
import kagglehub
path = kagglehub.dataset_download("moltean/fruits")
print("Path to dataset files:", path)

Path to dataset files: C:\Users\Stephen\.cache\kagglehub\datasets\moltean\fruits\versions\45


In [36]:
# Define number of classes for your specific task
# Change this to your actual number of classes (1000 for ImageNet)
num_classes = 1000  

In [37]:
# Create input layers with appropriate sizes
xception_input = Input(shape=(299, 299, 3), name='xception_input')
densenet_input = Input(shape=(224, 224, 3), name='densenet_input')

In [38]:
# Load pre-trained models without top layers
xception_base = Xception(weights='imagenet', include_top=False, input_tensor=xception_input)
densenet_base = DenseNet201(weights='imagenet', include_top=False, input_tensor=densenet_input)

In [39]:
# Add global average pooling to both models
x_features = GlobalAveragePooling2D(name='xception_gap')(xception_base.output)
d_features = GlobalAveragePooling2D(name='densenet_gap')(densenet_base.output)

In [ ]:
# Combine features from both models
combined = Concatenate(name='feature_fusion')([x_features, d_features])

In [ ]:
# Add final classification layer
predictions = Dense(num_classes, activation='softmax', name='predictions')(combined)

In [ ]:
# Create ensemble model
ensemble_model = Model(inputs=[xception_input, densenet_input], outputs=predictions)

In [ ]:
# Print model summary
# ensemble_model.summary()

In [ ]:
# Compile the model
ensemble_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
print(f"Model created successfully with {num_classes} output classes")
print(f"Total parameters: {ensemble_model.count_params():,}")

In [ ]:
# tf.keras.utils.plot_model(ensemble_model, show_shapes=True)

In [ ]:
# Extract features from specific layers
xception_features = xception_base.get_layer('block14_sepconv2_act').output  # Shape: (10, 10, 2048)
densenet_features = densenet_base.get_layer('pool4_pool').output  # Shape: (7, 7, 1664)

In [ ]:
# Resize to match dimensions
from tensorflow.keras.layers import UpSampling2D, Conv2D
densenet_upsampled = UpSampling2D(size=(2, 2))(densenet_features)
densenet_adjusted = Conv2D(2048, 1, activation='relu')(densenet_upsampled)

In [ ]:
# Combine features
combined_features = tf.keras.layers.Add()([xception_features, densenet_adjusted])